In [4]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import os
import json
import datetime
import pickle

stemmer = LancasterStemmer()

In [5]:
training_data = []

with open("libreto.dat", "rb") as f:
    training_data = pickle.load (f)

print ("%s sentences in training data" % len(training_data))

3643 sentences in training data


In [6]:
words = []
classes = []
documents = []
ignore_words = ['?']
for pattern in training_data:
    w = nltk.word_tokenize(pattern['sentence'])
    words.extend(w)
    #Agrega documentos en el corpus
    documents.append((w, pattern['class']))
    # Agrega a la lista de clase
    if pattern['class'] not in classes:
        classes.append(pattern['class'])

# Stop words
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = list(set(words))

#duplicados
classes = list(set(classes))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

3643 documents
5 classes ['raj', 'howard', 'leonard', 'penny', 'sheldon']
3951 unique stemmed words ['humourmomet', 'battlest', 'addict', 'year', 'peek', 'pretty', 'issu', 'ha-ha-h', 'pseudo', 'respect', 'refresh', 'booz', 'res', 'tal', 'cow', 'splintered', 'twin', 'isaac', 'repuls', 'model', 'pap', 'respons', 'knock', 'send', 'peer', 'hulk', 'uncal', 'yakkety-yakking', 'workshop', 'arm', 'nors', 'sucky', 'per', 'nurs', 'ir', 'pen', 'celsi', 'keen', 'tornado', 'culin', 'graph', 'substitut', 'suburb', 'serotonin', 'glow', 'rog', 'yearn', 'trunk', 'you…', 'who', 'cardio-funk', 'polym', 'bowel', 'sex', 'sportsm', 'lobst', 'legend', 'unfortun', 'daught', 'anyth', 'sociopa', 'carefr', 'mail-order', 'hard', 'despit', 'blah', 'finkleday', 'hunch', 'ss', 'myself', 'profil', 'humbl', 'sight', 'exceiv', 'anoth', 'cross', 'fork', 'engin', 'pour', 'hair', 'four', 'wor', 'beam', 'beatbox', 'diet', 'he', 'chex', 'outweigh', 'cutest', 'scor', 'six-thirty', 'list', '5.0', 'prowl', 'today', 'picasso', 

In [7]:
#Datos de entrenamiento
training = []
output = []
output_empty = [0] * len(classes)

#Bolsa de palabras para cada oracion
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # matriz de palabras
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    training.append(bag)
    # 1 para la etiqueta actual en otro caso 0
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    output.append(output_row)

print ("# words", len(words))
print ("# classes", len(classes))

# words 3951
# classes 5


In [8]:
i = 0
w = documents[i][0]
print ([stemmer.stem(word.lower()) for word in w])
print (training[i])
print (output[i])

['so', 'if', 'a', 'photon', 'is', 'direct', 'through', 'a', 'plan', 'with', 'two', 'slit', 'in', 'it', 'and', 'eith', 'slit', 'is', 'observ', 'it', 'wil', 'not', 'go', 'through', 'both', 'slit', '.', 'if', 'it', '’', 's', 'unobserv', 'it', 'wil', ',', 'howev', ',', 'if', 'it', '’', 's', 'observ', 'aft', 'it', '’', 's', 'left', 'the', 'plan', 'but', 'bef', 'it', 'hit', 'it', 'target', ',', 'it', 'wil', 'not', 'hav', 'gon', 'through', 'both', 'slit', '.']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [9]:
import numpy as np
import time

#valores entre 0 y uno
def sigmoid(x):
    output = 1/(1+np.exp(-x))
    return output

# derivada de sigmoid
def sigmoid_output_to_derivative(output):
    return output*(1-output)
 
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

#Bolsa de palabras
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

def think(sentence, show_details=False):
    x = bow(sentence.lower(), words, show_details)
    if show_details:
        print ("sentence:", sentence, "\n bow:", x)
    l0 = x
    l1 = sigmoid(np.dot(l0, synapse_0))
    l2 = sigmoid(np.dot(l1, synapse_1))
    return l2

In [10]:
def train(X, y, hidden_neurons=10, alpha=1, epochs=50000, dropout=False, dropout_percent=0.5):

    print ("Training with %s neurons, alpha:%s, dropout:%s %s" % (hidden_neurons, str(alpha), dropout, dropout_percent if dropout else '') )
    print ("Input matrix: %sx%s    Output matrix: %sx%s" % (len(X),len(X[0]),1, len(classes)) )
    np.random.seed(1)

    last_mean_error = 1
    # Los pesos se inicializan de forma aleatoria
    synapse_0 = 2*np.random.random((len(X[0]), hidden_neurons)) - 1
    synapse_1 = 2*np.random.random((hidden_neurons, len(classes))) - 1

    prev_synapse_0_weight_update = np.zeros_like(synapse_0)
    prev_synapse_1_weight_update = np.zeros_like(synapse_1)

    synapse_0_direction_count = np.zeros_like(synapse_0)
    synapse_1_direction_count = np.zeros_like(synapse_1)
        
    for j in iter(range(epochs+1)):

        # Capas
        layer_0 = X
        layer_1 = sigmoid(np.dot(layer_0, synapse_0))
                
        if(dropout):
            layer_1 *= np.random.binomial([np.ones((len(X),hidden_neurons))],1-dropout_percent)[0] * (1.0/(1-dropout_percent))

        layer_2 = sigmoid(np.dot(layer_1, synapse_1))

        # Incertidumbre
        layer_2_error = y - layer_2

        if (j% 10000) == 0 and j > 5000:
            if np.mean(np.abs(layer_2_error)) < last_mean_error:
                print ("delta after "+str(j)+" iterations:" + str(np.mean(np.abs(layer_2_error))) )
                last_mean_error = np.mean(np.abs(layer_2_error))
            else:
                print ("break:", np.mean(np.abs(layer_2_error)), ">", last_mean_error )
                break
                
        layer_2_delta = layer_2_error * sigmoid_output_to_derivative(layer_2)

        layer_1_error = layer_2_delta.dot(synapse_1.T)


        layer_1_delta = layer_1_error * sigmoid_output_to_derivative(layer_1)
        
        synapse_1_weight_update = (layer_1.T.dot(layer_2_delta))
        synapse_0_weight_update = (layer_0.T.dot(layer_1_delta))
        
        if(j > 0):
            synapse_0_direction_count += np.abs(((synapse_0_weight_update > 0)+0) - ((prev_synapse_0_weight_update > 0) + 0))
            synapse_1_direction_count += np.abs(((synapse_1_weight_update > 0)+0) - ((prev_synapse_1_weight_update > 0) + 0))        
        
        #Al experimentar con diferentes 'alfa' 
        #(parámetro de descenso de gradiente) 
        #y ver cómo afecta la tasa de error. 
        #Este parámetro ayuda a nuestro ajuste 
        #de error a encontrar la tasa de error más baja
        synapse_1 += alpha * synapse_1_weight_update
        synapse_0 += alpha * synapse_0_weight_update
        
        prev_synapse_0_weight_update = synapse_0_weight_update
        prev_synapse_1_weight_update = synapse_1_weight_update

    now = datetime.datetime.now()

    synapse = {'synapse0': synapse_0.tolist(), 'synapse1': synapse_1.tolist(),
               'datetime': now.strftime("%Y-%m-%d %H:%M"),
               'words': words,
               'classes': classes
              }
    synapse_file = "synapses.json"

    with open(synapse_file, 'w') as outfile:
        json.dump(synapse, outfile, indent=4, sort_keys=True)
    print ("saved synapses to:", synapse_file)


In [11]:
X = np.array(training)
y = np.array(output)

start_time = time.time()

train(X, y, hidden_neurons=20, alpha=0.1, epochs=100000, dropout=False, dropout_percent=0.2)

elapsed_time = time.time() - start_time
print ("processing time:", elapsed_time, "seconds")

Training with 20 neurons, alpha:0.1, dropout:False 
Input matrix: 3643x3951    Output matrix: 1x5
delta after 10000 iterations:0.2784518253989951
delta after 20000 iterations:0.27845182539883173
delta after 30000 iterations:0.2784518253986658
delta after 40000 iterations:0.27845182539849733
delta after 50000 iterations:0.2784518253983261
delta after 60000 iterations:0.27845182539815216
delta after 70000 iterations:0.2784518253979755
delta after 80000 iterations:0.27845182539779595
delta after 90000 iterations:0.2784518253976135
delta after 100000 iterations:0.278451825397428
saved synapses to: synapses.json
processing time: 21943.582069158554 seconds


In [12]:

ERROR_THRESHOLD = 0.2
synapse_file = 'synapses.json' 
with open(synapse_file) as data_file: 
    synapse = json.load(data_file) 
    synapse_0 = np.asarray(synapse['synapse0']) 
    synapse_1 = np.asarray(synapse['synapse1'])

def classify(sentence, show_details=False):
    results = think(sentence, show_details)

    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD ] 
    results.sort(key=lambda x: x[1], reverse=True) 
    return_results =[[classes[r[0]],r[1]] for r in results]
    print ("%s \n classification: %s" % (sentence, return_results))
    return return_results

classify("You want to hear an interesting thing about stairs?")#sheldon
classify("I don’t care. Two millimetres? That doesn’t seem right.")#leonard
classify("Oh, you’re inviting me over to eat?")#penny
classify("Wait till you see this.")#howard
classify("It’s fantastic. Unbelievable.")#raj
print ()
classify("It’s before he became a creepy computer voice:.", show_details=True)#howard

sudo make me a sandwich 
 classification: [['sheldon', 0.9999999999938913]]
how are you today? 
 classification: [['sheldon', 0.9999999999239289]]
talk to you tomorrow 
 classification: [['sheldon', 0.9999999995552937]]
who are you? 
 classification: [['sheldon', 0.9999999999615465]]
make me some lunch 
 classification: [['sheldon', 0.9999999999993365]]

found in bag: how
found in bag: was
found in bag: yo
found in bag: lunch
sentence: how was your lunch? 
 bow: [0 0 0 ... 0 0 0]
how was your lunch? 
 classification: [['sheldon', 0.9999999989413881]]


[['sheldon', 0.9999999989413881]]